In [40]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
import pandas as pd

def load_data():
    col_names = [
                 'timestamp'
                ,'street_shoe_size'
                ,'fav_sneaker_model'
                ,'foot_len'
                ,'foot_width'
                ,'foot_type'
                ,'foot_deform'
                ,'shoe_brand'
                ,'shoe_model'
                ,'shoe_model_manual'
                ,'climb_shoe_size'
                ,'shoe_age'
                ,'favourite'
                ,'vorspann'
                ,'downturn'
                ,'asymetry'
                ,'closure_type'
                ,'climb_style'
                ,'terrain'
                ,'rate_fit'
                ,'rate_comfort'
                ,'rate_stiffness'
                ,'use_frequency'
                ,'climbed_grade'
                ,'overall_rating'
                ,'gender'
                ,'height'
                ,'weight'
                ,'age'
                ,'climbing_exp'
    ]
    
    data = pd.read_csv("./data/climb_shoe_survey.csv", header=0, names=col_names)

    return data

def get_brand_per_foottype(df_survey):
    '''
    Ermittle die meist genutzte Schuhmarke pro Fußtyp
    
    :param df: DataFrame df_survey
    
    :return: Dictionary Marken sortiert nach Anzahl
    '''
    foot_types = df_survey['foot_type'].unique()
    brand_per_foottype = pd.DataFrame(columns=['foot_type', 'brand', 'count'])

    for t in foot_types:
        new_row = {'foot_type' : t, 'brand' : df_survey['shoe_brand'][df_survey['foot_type'] == t].value_counts().keys()[0], 'count' : df_survey['shoe_brand'][df_survey['foot_type'] == t].value_counts()[0]}
        brand_per_foottype = brand_per_foottype.append(new_row, ignore_index=True)

    sorted_brands = brand_per_foottype.sort_values(by='count', ascending=False)
    
    return sorted_brands

def resolve_invalid_footdeforms(df_survey): # Resolve any observation with "no deform", but typed in a deform...
    '''
    Bereinige Angaben zur Fehlstellung. Alle Beobachtungen, die sowohl Nein als auch eine weitere Angabe gemacht haben.
    
    :param df: DataFrame df_survey
    
    :return: DataFrame df
    '''

    for i in df_survey[df_survey['foot_deform'] == 'Nein, Spreizfuß'].index:
        df_survey.loc[i, 'foot_deform'] = 'Spreizfuß'
    for i in df_survey[df_survey['foot_deform'] == 'Nein, Knickfuß'].index:
        df_survey.loc[i, 'foot_deform'] = 'Knickfuß'
    for i in df_survey[df_survey['foot_deform'] == 'Nein, Senkfuß, Spreizfuß'].index:
        df_survey.loc[i, 'foot_deform'] = 'Senkfuß, Spreizfuß'
    for i in df_survey[df_survey['foot_deform'] == 'Nein, Hallux Valgus, Senkfuß'].index:
        df_survey.loc[i, 'foot_deform'] = 'Hallux Valgus, Senkfuß'
    for i in df_survey[df_survey['foot_deform'] == 'Nein, Hallux Valgus'].index:
        df_survey.loc[i, 'foot_deform'] = 'Hallux Valgus'
    for i in df_survey[df_survey['foot_deform'] == 'Nein, Senkfuß'].index:
        df_survey.loc[i, 'foot_deform'] = 'Senkfuß'
    for i in df_survey[df_survey['foot_deform'] == 'Nein, Senkfuß, Spreizfuß'].index:
        df_survey.loc[i, 'foot_deform'] = 'Senkfuß, Spreizfuß'
    for i in df_survey[df_survey['foot_deform'] == 'Nein, Plattfuß'].index:
        df_survey.loc[i, 'foot_deform'] = 'Plattfuß'
     
    return df_survey

def translate_values(df_survey, search_col, search_str, replace_str):
    '''
    Transformation und Bereinigung von Text. Ersetzt einen String-Wert gegen einen anderen.
    Zusätzlich wird der Wert kleingeschrieben zurückgegeben.
    
    :param df: DataFrame df_survey
    :param search_col: String relevante Spalte
    :param search_str: String Such-String
    :param replace_str: String Ersatz-String
    
    :return: DataFrame df
    '''
    df = df_survey[search_col].str.replace(search_str, replace_str).apply(lambda x: x.lower())
    return df


def resolve_invalid_foottype(df_survey, valid_foottype, false_foottype):
    '''
    Bereinige ungültige Fußtypen, z.B. Freitextangaben
    
    :param df: DataFrame df_survey
    :param col: String Spalte, die umgewandelt werden soll
    
    :return: DataFrame df_survey
    '''
    df = df_survey.apply(
        lambda row: valid_foottype
            if row.foot_type == false_foottype
          else row.foot_type, axis=1)
    return df


def generate_dummies(df_survey, col):
    '''
    Erstelle Dummy-Variablen für Spalten mit Komma-getrennten Listen als Ausprägungen
    
    :param df: DataFrame df_survey
    :param col: String Spalte, die umgewandelt werden soll
    
    :return: DataFrame df_survey
    '''
    df_survey = pd.concat([df_survey.drop(col, 1), df_survey[col].str.get_dummies(sep=", ")], 1)
    
    return df_survey

def count_deforms(df_survey): 
    '''
    Zähle pro Fehlstellung alle Vorkommnisse
    
    :param df: DataFrame df_survey
    :return: Dictionary mit sortierten Anzahlen pro Fehlstellung
    '''
    deformations = ['hallux', 'senkfuß','spreizfuß','plattfuß', 'hohlfuß','knickfuß']
    stats = {}

    for deform in deformations:
        stats.update({deform : df_survey[deform.lower().split()[0]].value_counts()[1]})
    print(stats) 
    stats_sorted = dict(sorted(stats.items(),key= lambda x:x[1]))
    
    return stats_sorted


def resolve_shoemodels(df_survey, models):
    '''
    Übernehme alle manuell eingetragenen Schuhmodelle
    
    :param df: DataFrame df_survey
    :param models: Dictionary mit Index und Schuhmodell
    
    :return: DataFrame df_survey
    '''
    for i in models:
        df_survey.loc[i,'shoe_model'] = models[i]
    return df_survey

In [42]:
# Daten laden
df_survey = load_data()

In [43]:
'''
PREPROCESSING
Daten vorbereiten
'''

# Ungültige Beobachtungen löschen
for i in df_survey[df_survey['foot_deform'] == 'Nein, abgeflachtes Quergewölbe, aber noch kein Spreizfuß'].index:
        df_survey.drop(index=i, inplace=True, axis=0)
        
df_survey.reset_index(drop=True, inplace=True)


# Daten transformieren:  - Ausprägungen für Fehlstellungen bereinigen, 
#                          wenn 'Nein' und trotzdem weitere Fehlstellungen aus gewählt wurden
#                        - Ausprägungen umbenennen
df_survey              = resolve_invalid_footdeforms(df_survey[:])
df_survey['foot_type'] = resolve_invalid_foottype(df_survey[:], 'Ägyptischer Fußtyp', 'Mix aus Römisch und Ägyptisch')
df_survey['deform']    = translate_values(df_survey[:], 'foot_deform', ' Valgus', '')
df_survey['foottype']  = translate_values(df_survey[:], 'foot_type', ' Fußtyp', '')

'''
Sonstige genannte Schuhmodelle übernehmen
'Sonstige' Schuhmodelle imputieren
'''
# Dictionary mit bekanntem Index und Schuhmodell. Dieser wird aufbereitet im Dictonary gespeichert
models = {77  : 'Boreal - SATORI',
          31  : 'Evolv - SHAMAN 2',
          86  : 'Evolv - SHAMAN 2',
          88  : 'Evolv - SHAMAN 2',
          103 : 'La Sportiva - MILLET EASY UP',
          140 : 'Red Chili - AMP',
          154 : 'Evolv - SHAMAN 2',
          155 : 'Evolv - SHAMAN 2'}

df_survey = resolve_shoemodels(df_survey[:], models)

# Ermittle das häufigste Schuhmodell einer Marke und ersetze 'Sonstige'
most_common_models = {}
    
for i in df_survey.groupby(['shoe_brand', 'shoe_model'])['shoe_model'].count().sort_values().groupby(level=0).tail(1).index:
    most_common_models.update({i[0]:i[1]})
for i in df_survey['shoe_brand'][df_survey['shoe_model'] == 'Sonstige'].index:
    df_survey.at[i,'shoe_model'] = most_common_models[df_survey['shoe_brand'].iloc[i]]

df_survey.at[18,'shoe_model'] = df_survey.at[18,'shoe_brand']

# One hot encoding von Spalten mit komma-separierten Ausprägungslisten
df_survey = generate_dummies(df_survey[:], 'deform')
df_survey = generate_dummies(df_survey[:], 'climb_style')
df_survey = generate_dummies(df_survey[:], 'terrain')
df_survey = generate_dummies(df_survey[:], 'foottype')
df_survey = pd.get_dummies(data=df_survey, columns=['gender'],drop_first=True)

df_survey.head()

,timestamp,street_shoe_size,fav_sneaker_model,foot_len,foot_width,foot_type,foot_deform,shoe_brand,shoe_model,shoe_model_manual,...,Bouldern,Mehrseillängen,Sportklettern,Beides,Halle,Outdoor,griechischer,römischer,ägyptischer,gender_Weiblich
0,24.02.2021 22:27:44,41.5,Vans Authentic,25.5,10.0,Ägyptischer Fußtyp,Nein,La Sportiva,La Sportiva - SKWAMA,NaN,...,1,0,1,1,0,0,0,0,1,0
1,25.02.2021 08:26:50,38.5,Adidas cloudfoam,24.2,9.8,Römischer Fußtyp,Hallux Valgus,La Sportiva,La Sportiva - SOLUTION COMP WOMAN,NaN,...,1,0,0,0,1,0,0,1,0,1
2,25.02.2021 10:28:25,46.0,Black Diamond Mission LT,29.0,10.0,Griechischer Fußtyp,Nein,La Sportiva,La Sportiva - SKWAMA,NaN,...,1,0,1,1,0,0,1,0,0,0
3,25.02.2021 10:41:47,46.0,Black Diamond Mission LT,29.0,10.0,Griechischer Fußtyp,Nein,La Sportiva,La Sportiva - SKWAMA,NaN,...,1,0,1,1,0,0,1,0,0,0
4,25.02.2021 11:35:49,42.0,Adidas Stan Smith,27.0,10.0,Römischer Fußtyp,Hallux Valgus,Tenaya,Tenaya - OASI,NaN,...,1,0,1,0,1,0,0,1,0,0


In [44]:
'''
Erzeugung Dataframe mit aufbereiteten Daten für die Zuführung ins Modell-Training. 
'''
#Hier wird die Zielvariable shoe_model bereits entfernt!
df_prep = df_survey.drop(['timestamp','foot_deform','foot_type', 'fav_sneaker_model', 'shoe_brand', 'shoe_model',
       'shoe_model_manual', 'climb_shoe_size', 'shoe_age', 'favourite',
       'vorspann', 'downturn', 'asymetry', 'closure_type','rate_fit', 'rate_comfort', 'rate_stiffness',
       'use_frequency', 'climbed_grade', 'overall_rating', 'Beides', 'nein'], axis=1)
df_prep.head(1)

,street_shoe_size,foot_len,foot_width,height,weight,age,climbing_exp,fersenbein,hallux,hohlfuß,...,spreizfuß,Bouldern,Mehrseillängen,Sportklettern,Halle,Outdoor,griechischer,römischer,ägyptischer,gender_Weiblich
0,41.5,25.5,10.0,168,61,29,4.0,0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [45]:
df_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   street_shoe_size  181 non-null    float64
 1   foot_len          181 non-null    float64
 2   foot_width        181 non-null    float64
 3   height            181 non-null    int64  
 4   weight            181 non-null    int64  
 5   age               181 non-null    int64  
 6   climbing_exp      181 non-null    float64
 7   fersenbein        181 non-null    int64  
 8   hallux            181 non-null    int64  
 9   hohlfuß           181 non-null    int64  
 10  knickfuß          181 non-null    int64  
 11  plattfuß          181 non-null    int64  
 12  senkfuß           181 non-null    int64  
 13  spreizfuß         181 non-null    int64  
 14  Bouldern          181 non-null    int64  
 15  Mehrseillängen    181 non-null    int64  
 16  Sportklettern     181 non-null    int64  
 1

In [46]:
'''
Ausreißer bereinigen
'''

# Fußlänge unter 20
mask = df_prep['foot_len'] < 20
df_prep.loc[mask, 'foot_len'] = round(df_prep['foot_len'].mean(),2)

In [47]:
df_prep.shape

(181, 23)

In [56]:
''' Dimensionsreduktion mittels Korrelationsmatrix
'''
corr_matrix = df_prep.corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),
                                 k=1).astype(np.bool))

to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]

df_prep.drop(df_prep.columns[to_drop], axis=1, inplace=True)
df_prep.shape

(181, 23)

In [15]:
'''
Schuhklassifikation
'''

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Datensatz in Training- und Testdaten splitten
X = df_prep
y= df_survey['shoe_model']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.04)

# Trainingsdaten skalieren

scaled_features = StandardScaler().fit_transform(X_train)

In [31]:
'''
Dimensionsreduktion mittels Hauptkomponentenanalyse (PCA)
'''

from sklearn.decomposition import PCA

pca = PCA(n_components=0.95, whiten=True)

pca_features = pca.fit_transform(scaled_features)

print("Ursprüngliche Anzahl an Merkmalen:", scaled_features.shape[1])
print("Reduzierte Anzahl an Merkmalen:", pca_features.shape[1])

Ursprüngliche Anzahl an Merkmalen: 23
Reduzierte Anzahl an Merkmalen: 17


In [30]:
'''
Dimensionsreduktion mitels Matrixfaktorisierung
'''

from sklearn.decomposition import NMF

nmf = NMF(n_components=5, random_state=1, max_iter=32000)
nmf_features = nmf.fit_transform(X_train)

print("Ursprüngliche Anzahl an Merkmalen:", X_train.shape[1])
print("Reduzierte Anzahl an Merkmalen:", nmf_features.shape[1])

Ursprüngliche Anzahl an Merkmalen: 23
Reduzierte Anzahl an Merkmalen: 5
